<a href="https://colab.research.google.com/github/IverMartinsen/deeplearning/blob/master/FCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title import modules { form-width: "20%" }
import sys
from google.colab import drive

drive.mount('/content/gdrive')
sys.path.append('/content/gdrive/MyDrive/DL home exam/Problem 2/problem2/DataLoaders/')

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout
from tensorflow.keras import Model
from tensorflow.keras.metrics import MeanIoU
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap as lcm
from camvid_loader import load_camvid

Mounted at /content/gdrive


In [ ]:
#@title load data { form-width: "20%" }
# load data and move axis
x_tr, y_tr, x_te, y_te, _ = load_camvid('/content/gdrive/MyDrive/DL home exam/Problem 2/problem2/data/')
x_tr = tf.image.resize(np.moveaxis(x_tr, 1, -1), (180, 240))
y_tr = tf.image.resize(np.moveaxis(y_tr, 1, -1), (180, 240))
x_te = tf.image.resize(np.moveaxis(x_te, 1, -1), (180, 240))
y_te = tf.image.resize(np.moveaxis(y_te, 1, -1), (180, 240))

# number of classes and image size
num_class = 12
im_size = x_tr.shape[-3:-1]

In [ ]:
#@title define model { form-width: "20%" }
class FCN(Model):
    def __init__(self, num_class, imsize):
        super().__init__()

        self.imsize = imsize

        self.conv1 = Conv2D(64, 3, padding='same', activation='relu')
        self.conv2 = Conv2D(64, 3, padding='same', activation='relu')
        self.pool1 = MaxPool2D()

        self.conv3 = Conv2D(128, 3, padding='same', activation='relu')
        self.conv4 = Conv2D(128, 3, padding='same', activation='relu')
        self.pool2 = MaxPool2D()

        self.conv5 = Conv2D(256, 3, padding='same', activation='relu')
        self.conv6 = Conv2D(256, 3, padding='same', activation='relu')
        self.pool3 = MaxPool2D()

        self.conv7 = Conv2D(512, 3, padding='same', activation='relu')
        self.conv8 = Conv2D(512, 3, padding='same', activation='relu')
        self.pool4 = MaxPool2D()

        self.conv9 = Conv2D(4096, 1, activation='relu')
        self.drop1 = Dropout(0.5)
        self.conv10 = Conv2D(4096, 1, activation='relu')
        self.drop2 = Dropout(0.5)

        self.conv11 = Conv2D(num_class, 1, activation='softmax')

    def call(self, x):

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.pool1(x)

        x = self.conv3(x)
        x = self.conv4(x)
        x = self.pool2(x)
        
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.pool3(x)

        x = self.conv7(x)
        x = self.conv8(x)
        x = self.pool4(x)

        x = self.drop1(self.conv9(x))
        x = self.drop2(self.conv10(x))
        x = self.conv11(x)

        return tf.image.resize(x, self.imsize)

    def segment(self, x):
        return np.argmax(self.predict(x), axis=3) / (num_class-1)

    def accuracy(self, images, labels):
        return (np.sum(np.argmax(model.predict(images),
                                 axis = 3)[..., np.newaxis]
                       == labels) / np.prod(labels.shape))
    
    def certainty(self, x):
        return np.max(self.predict(x), axis=3)

In [ ]:
#@title fit model and display loss { form-width: "20%" }
model = FCN(num_class, im_size)

model.compile(tf.keras.optimizers.Adam(),
              tf.keras.losses.SparseCategoricalCrossentropy())

callbacks=[tf.keras.callbacks.EarlyStopping(patience = 10,
                                            restore_best_weights=True)]

history = model.fit(x_tr,
                    y_tr,
                    epochs=1000,
                    validation_split=0.2,
                    callbacks=callbacks)

plt.figure(figsize=(12, 8))
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

Epoch 1/1000
 4/12 [=========>....................] - ETA: 6:21 - loss: 3.2133

In [ ]:
#@title compute accuracy { form-width: "20%" }

images = x_te
labels = y_te

# class wise accuracy
acc = np.zeros(num_class)

for i in range(num_class):
    count = np.array(np.where(labels == i)).shape[1]
    acc[i] = np.sum(np.argmax(model.predict(images),
                              axis = 3)[..., np.newaxis][np.where(labels == i)]
                    == i) / count

# total accuracy
act = (np.sum(np.argmax(model.predict(images), axis = 3)[..., np.newaxis] 
              == labels) / np.prod(labels.shape))

In [ ]:
#@title compute IOU { form-width: "20%" }

IoU = MeanIoU(num_class)
IoU.update_state(y_te, np.argmax(model.predict(x_te), axis = 3)[..., np.newaxis])
IoU.result()

In [ ]:
#@title class names { form-width: "20%" }
class_names = ['sky', 'buildings', 'poles', 'road',
               'sidewalk', 'trees', 'sign details', 'plants',
               'cars', 'pedestrians', 'cyclists', 'white stripes']

In [ ]:
#@title create colormap { form-width: "20%" }

colors = np.zeros((12, 3))

colors[(0, 1, 3, 4, 9, 10), :] = np.array(
    cm.get_cmap('tab20').colors)[(1, 16, 15, 14, 6, 2), :]
colors[2, :] = np.zeros(3)
colors[11, :] = np.ones(3)
colors[5:8, :] = np.array(cm.get_cmap('tab20c').colors)[9:12, :]
colors[8, :] = np.array(cm.get_cmap('tab20b').colors)[0, :]
colors[6, :] = np.array(cm.get_cmap('Accent').colors)[4, :]

custom_cm = lcm(colors)

In [ ]:
#@title segment and display random images from test set { form-width: "20%" }
sample = np.random.choice(y_te.shape[0], 3, replace=False)
images = model.segment(x_tr[sample])

fig, axes = plt.subplots(3, 2, figsize=(12, 13.5))

for i, ax in enumerate(axes):
    ax[0].imshow(x_tr[sample[i]])
    ax[0].axis('off')   
    ax[1].imshow(y_tr[sample[i]][:, :, 0], cmap=custom_cm)
    ax[1].axis('off')

In [ ]:
#@title display certainty maps { form-width: "20%" }
fig, axes = plt.subplots(3, 2, figsize=(12, 13.5))

for i, ax in enumerate(axes):
    ax[0].imshow(x_tr[sample[i]], 'grey')
    ax[0].axis('off')